<a href="https://colab.research.google.com/github/nrohrbach/sharedmobility_monitoring/blob/main/sharedmobility_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sharedmobility.ch - Validation of GBFS-Feeds

In [1]:
pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 769.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 4.0 MB/s eta 0:00:00


In [2]:
import requests
import pandas as pd
from datetime import datetime

In [3]:
# GBFS Feeds abfragen
url = 'https://gbfs.prod.sharedmobility.ch/v2/gbfs'
feeds = requests.get(url).json()

In [5]:
# Alle GBFS Feeds als Dataframe laden
id = [s['id'] for s in feeds['systems']]
url = [s['url'] for s in feeds['systems']]
gbfsfeeds = pd.DataFrame(list(zip(id, url)),columns =['provider', 'gbfsurl'])
providers = list(gbfsfeeds['provider'])

In [16]:
# Nur einen Provider temporär einstellen!
providers = ['bolt_zurich','velospot']
providers

['bolt_zurich', 'velospot']

In [21]:
# Validieren!
JsonResponse = []
providerList2 = []
HasErrors = []
NumberOfErrors = []

for provider in providers:
    try:
      headers = {
          'accept': '*/*',
          'Content-Type': 'application/json',
          'User-Agent': 'geoinformation@bfe.admin.ch',
      }

      json_data = {
          'url': 'https://gbfs.prod.sharedmobility.ch/v2/gbfs/'+ provider +'/gbfs',
          'options': {
              'freefloating': False,
              'docked': False,
              'version': None,
              'auth': {
                  'type': None,
                  'basicAuth': {
                      'user': None,
                      'password': None,
                  },
                  'bearerToken': {
                      'token': None,
                  },
                  'oauthClientCredentialsGrant': {
                      'user': None,
                      'password': None,
                      'tokenUrl': None,
                  },
              },
          },
      }
      response = requests.post('https://gbfs-validator.netlify.app/.netlify/functions/validator', headers=headers, json=json_data).json()

      providerList2.append(provider)
      JsonResponse.append(response)
      #HasErrors.append(response['summary']['hasErrors'])
      #NumberOfErrors.append(response['summary']['errorsCount'])

    except:
      providerList2.append(provider)
      JsonResponse.append(response)
      #HasErrors.append('nan')
      #NumberOfErrors.append('nan')

In [18]:
# Dataframe erstellen
dict = {'Provider': providerList2, 'Fehlerhaft': HasErrors, 'AnzahlFehler': NumberOfErrors}
dfValidate = pd.DataFrame(dict)

ValueError: ignored

In [22]:
# Dataframe erstellen
dict = {'Provider': providerList2, 'Response': JsonResponse}
dfValidate = pd.DataFrame(dict)

In [23]:
dfValidate

,Provider,Response
0,bolt_zurich,{'errorMessage': '2023-07-25T07:35:45.895Z 928...
1,velospot,{'errorMessage': '2023-07-25T07:36:12.293Z 4e3...
